In [1]:
#spark
__author__ = 'vcoder'

import os
import sys

#change them when spark and pyspark path changes

#SPARK_HOME = "/home/vcoder/EDA/spark-1.5.0"
#SPARK_HOME_PYTHON = SPARK_HOME + "/python"

SPARK_HOME = "/home/worker/software/spark-1.5.0"
SPARK_HOME_PYTHON = SPARK_HOME + "/python"

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME_PYTHON)

from pyspark import SparkContext
from pyspark import SparkConf

In [2]:
import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def ip2city(ip):
    try:
        city = reader.city(ip).city.name
    except:
        city = 'not found'
    return city

def ip2lalo(ip):
    try:
        la = reader.city(ip).location.latitude
        lo = reader.city(ip).location.longitude
    except:
        la = 0.
        lo = 0.
    return (la, lo)


In [3]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [4]:
sc = SparkContext('local', 'ip2geo')

In [5]:
raw = sc.textFile('csv/output.csv')
#ipAndGeo = raw.map(lambda x: x.split(',')).map(lambda x: ((x[0],ip2city(x[0])), (x[1],ip2city(x[1]))))

In [19]:
#print ipAndGeo.collect()[0:1]
print raw.map(lambda x: x[2]).distinct().collect()

[u'.', u'1', u'0', u'3', u'2', u'5', u'4', u'7', u'6', u'9', u'8']


In [18]:
print ip2city('51.173.229.255')

None


In [12]:
raw2 = raw.map(lambda x: x.split(','))
print raw2.collect()[0:4]

[[u'0.000000000', u'51.173.229.255', u'71.126.222.64', u'1'], [u'0.000004000', u'51.173.229.255', u'71.126.222.64', u'1'], [u'0.001440000', u'39.247.10.192', u'71.126.222.64', u'1'], [u'0.001445000', u'71.126.222.64', u'39.247.10.192', u'1']]


In [15]:
print raw2.map(lambda x: x[2]).distinct().count()

31


In [9]:
raw3 = raw2.map(lambda x: (x[1], x[2]))
print raw3.collect()[0][0]

51.173.229.255


In [11]:
#raw4 = raw3.map(lambda x: ip2lalo(x[0]))
#print raw4.collect()[0]
info = map(lambda x: ip2city(x[0]), raw3.collect()[0:10000])
# for ips in raw3.collect():
#     city = ip2city(ips[0])
#     lalo = ip2lalo(ips[0])
#     info.append((city, lalo))

raw4 = sc.parallelize(info);
print raw4.collect()[0:9]

[None, None, None, None, u'Palo Alto', u'Montreal', u'Honiara', u'Montreal', None]


In [ ]:
print ip2lalo(u'128.101.101.101')
print type(reader.city('128.101.101.101').city.name)

In [16]:
src = raw3.map(lambda x: x[0]).take(1000)
#print src[0:9]
srcAndLoc = map(lambda x: [x, ip2city(x), ip2lalo(x)], src)

In [18]:
print srcAndLoc[0:10]

[[u'51.173.229.255', None, (51.5, -0.13)], [u'51.173.229.255', None, (51.5, -0.13)], [u'39.247.10.192', None, (-6.175, 106.8286)], [u'71.126.222.64', None, (38.0, -97.0)], [u'192.120.148.227', u'Palo Alto', (37.3762, -122.1826)], [u'192.95.27.190', u'Montreal', (45.504, -73.5747)], [u'202.1.175.252', u'Honiara', (-9.4333, 159.95)], [u'192.95.27.190', u'Montreal', (45.504, -73.5747)], [u'51.173.229.255', None, (51.5, -0.13)], [u'192.120.148.227', u'Palo Alto', (37.3762, -122.1826)]]


In [43]:
aa = sc.parallelize([1,2,3])#.map(lambda x: (x,0))
bb = sc.parallelize([4,5,6,3])#.map(lambda x: (x,0))
cc = aa.union(bb)#.map(lambda (k,v): k)
print cc.sum()

24


In [38]:
print aa.take(2)

[1, 2]


In [40]:
print cc.top(5)

[6, 5, 4, 3, 3]
